# AI Companion with Memory

This Jupyter notebook implements an AI companion that engages in conversations and remembers previous interactions using OpenAI's GPT-4 model and a memory system.

## Overview

The AI Companion utilizes the following components:
- OpenAI's GPT-4 for natural language processing
- mem0ai for memory management
- Qdrant for vector storage
- Neo4j for graph storage

### Notes

*1. Ensure all necessary credentials and API keys are properly set up before running the notebook. Do not share sensitive information when pushing to a public repository.*

*2. This notebook is designed to run in Google Colab. Ensure that you have set up the necessary credentials in Google Colab's userdata before running the notebook.*

## Installing mem0ai and other dependencies

In [15]:
%%capture
!pip install mem0ai openai langchain-community rank-bm25 neo4j

# Importing Libraries

In [9]:
import os
import json
from google.colab import userdata
from openai import OpenAI
from mem0 import Memory
from typing import List, Dict
from mem0.memory.graph_memory import MemoryGraph

## Setting the OpenAI API keys

In [33]:
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')

## Setting Credentials

In [34]:
URL = userdata.get('URL')
USERNAME = userdata.get('USERNAME')
PASSWORD = userdata.get('PASSWORD')
QDRANT_COLLECTION = userdata.get('QDRANT_COLLECTION')
QDRANT_URL = userdata.get('QDRANT_URL')
QDRANT_API_KEY = userdata.get('QDRANT_API_KEY')
user_id=userdata.get('USER_ID')

# Setting Configurations

In [45]:
# Initialize the OpenAI client
client = OpenAI()

config = {
            "llm": {
                "provider": "openai",
                "config": {
                    "model": "gpt-4o",
                    "temperature": 0.2,
                }
            },
            "vector_store": {
        "provider": "qdrant",
        "config": {
            "collection_name": QDRANT_COLLECTION,
            "url": QDRANT_URL,
            "api_key": QDRANT_API_KEY
        }
    },
            "graph_store": {
                "provider": "neo4j",
                "config": {
                    "url": URL,
                    "username": USERNAME,
                    "password": PASSWORD
                },
            },
            "version": "v1.1"
        }

memory = Memory.from_config(config_dict=config)

**Checking the Memory**

In [ ]:
memory.get_all(user_id=user_id)

# AI Assistant

In [ ]:
class Companion:
    """
    A class representing an AI companion that can engage in conversations and remember previous interactions.
    """

    def __init__(self, client: OpenAI):
        """
        Initialize the Companion.

        :param client: An instance of the OpenAI client for API interactions.
        """
        self.client = client

    def ask(self, question: str, user_id: str) -> str:
        """
        Process a user's question and generate a response.

        :param question: The user's input question.
        :param user_id: The unique identifier for the user.
        :return: The AI-generated response to the question.
        """
        # Retrieve relevant previous memories
        previous_memories = memory.search(question, user_id=user_id)
        relevant_memories_text = "\n".join(mem["memory"] for mem in previous_memories['results'])

        # Construct the prompt with the question and relevant memories
        prompt = f"User input: {question}\nPrevious memories: {relevant_memories_text}"
        messages = [
            {
                "role": "system",
                "content": "You are the user's companion. Use the user's input and previous memories to respond. Answer based on the context provided."
            },
            {
                "role": "user",
                "content": prompt
            }
        ]

        try:
            # Generate a response using the OpenAI API
            stream = self.client.chat.completions.create(
                model="gpt-4",
                stream=True,
                messages=messages
            )

            answer = ""
            for chunk in stream:
                if chunk.choices[0].delta.content is not None:
                    content = chunk.choices[0].delta.content
                    print(content, end="", flush=True)
                    answer += content

            # Add the new interaction to memory
            memory.add(question, user_id=user_id)
            return answer
        except Exception as e:
            print(f"An error occurred: {e}")
            return ""

    def __enter__(self):
        """
        Enter the runtime context for the Companion.

        :return: The Companion instance.
        """
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        """
        Exit the runtime context for the Companion.

        :param exc_type: The exception type, if any.
        :param exc_val: The exception value, if any.
        :param exc_tb: The exception traceback, if any.
        """
        pass

def main():
    """
    The main function to run the AI companion interaction loop.
    """
    with Companion(client) as ai_companion:
        while True:
            text_input = input("\nEnter text (or 'quit' to exit):\t")
            if text_input.lower() == 'quit':
                print("Goodbye :)")
                break
            print("\nAssistant:")
            ai_companion.ask(text_input, user_id)

if __name__ == "__main__":
    main()

# Using Memory to Manage and Showcase Entities and Relations

In [66]:
# Wrap config in a class to provide attribute-style access
class ConfigWrapper:
    def __init__(self, config_dict):
        for key, value in config_dict.items():
            setattr(self, key, ConfigWrapper(value) if isinstance(value, dict) else value)

config_dict = {
    "graph_store": {
        "provider": "neo4j",
        "config": {
            "url": URL,
            "username": USERNAME,
            "password": PASSWORD
        },
        "llm": {
            "provider": "openai",
            "config": {
                "model": "gpt-4",
                "temperature": 0.7
            }
        }
    },
    "embedder": {
        "provider": "openai",
        "config": {
            "model": "gpt-4o",
        }
    },
    "llm": {
        "provider": "openai",
        "config": {
            "model": "gpt-4",
            "temperature": 0.7
        }
    }
}
config = ConfigWrapper(config_dict)
supported_embedder_keys = ["model", "api_key", "embedding_dims", "openai_base_url"]
config.embedder.config = {
    k: v for k, v in config_dict["embedder"]["config"].items() if k in supported_embedder_keys
}

# Convert the LLM config to a plain dictionary for compatibility
config.llm.config = config_dict["llm"]["config"]

memory = MemoryGraph(config)

text = "Alice, a data scientist, knows Bob, an AI researcher. Machine Learning includes Deep Learning."

entities = [
    {"type": "User", "name": "Alice", "attributes": {"description": "Data scientist"}},
    {"type": "User", "name": "Bob", "attributes": {"description": "AI researcher"}},
    {"type": "Topic", "name": "Machine Learning", "attributes": {}},
    {"type": "Topic", "name": "Deep Learning", "attributes": {}}
]

relationships = [
    {"source": "Alice", "relation": "KNOWS", "destination": "Bob"},
    {"source": "Machine Learning", "relation": "INCLUDES", "destination": "Deep Learning"}
]

# Adding entities to the memory graph using Cypher queries
for entity in entities:
    attributes_map = ", ".join([f"{key}: '{value}'" for key, value in entity["attributes"].items()])
    query = f"""
    MERGE (n:{entity['type']} {{name: '{entity['name']}'}})
    SET n += {{{attributes_map}}}
    """
    memory.graph.query(query)

# Adding relationships to the memory graph using Cypher queries
for relation in relationships:
    query = f"""
    MATCH (a {{name: '{relation['source']}'}}), (b {{name: '{relation['destination']}'}})
    MERGE (a)-[r:{relation['relation']}]->(b)
    """
    memory.graph.query(query)

# Querying Relationships
result = memory.graph.query("MATCH (n)-[r]->(m) RETURN n, r, m")
print("Graph Relationships:", result)


Graph Relationships: [{'n': {'user_id': 'aftar_ahmad', 'created': 1736439487777, 'name': 'aftar_ahmad', 'embedding': [0.02075430564582348, -0.03312419354915619, -0.029586657881736755, -0.037304915487766266, -0.03032173030078411, -0.015379088930785656, -0.037741366773843765, 0.024647891521453857, 0.01856057345867157, -0.002588545437902212, 0.03447948023676872, -0.029494773596525192, 0.035834770649671555, 0.005650867708027363, 0.033307962119579315, 0.010112985968589783, 0.018101153895258904, 0.00524313235655427, 0.020846189931035042, -0.0009245830588042736, 0.0220177099108696, 0.029862308874726295, 0.020191514864563942, -0.0188247412443161, 0.010440322570502758, -0.02517622336745262, -0.009768420830368996, 0.012622568756341934, 0.027128759771585464, -0.02917317859828472, -0.022626442834734917, -0.033905208110809326, 0.026669340208172798, 0.028598904609680176, -0.01719379983842373, -0.005484328139573336, 1.3425925317278598e-05, -0.028782671317458153, 0.003261883044615388, -0.0270598474889